In [1]:
# Библиотеки
from scipy.stats import chi2_contingency
import scipy.stats as stats

import pandas as pd
import numpy as np

# Загрузака и предобработка данных

In [637]:
# Обработка payments
payments = pd.read_csv('/Users/Josebaxter/Downloads/payments.csv', sep=';')

payments = payments.reset_index()
payments = payments.rename(columns=payments.iloc[0]).drop(payments.index[0])

payments.Amount = payments.Amount.astype('float')
payments.Balance = payments.Balance.astype('float')
payments['Payment Dt'] = pd.to_datetime(payments['Payment Dt'])

In [638]:
# Обработка payments sign-ups
sign = pd.read_csv('/Users/Josebaxter/Downloads/sign-ups.csv', sep=';')
sign = sign.reset_index()

# удаляем лишние символы
for i in sign.columns:
    sign[i] = sign[i].str.replace('\t','')

# делаем первую строку заголовком 
sign = sign.rename(columns=sign.iloc[0]).drop(sign.index[0])

# некоторые юзеры регистрировались повтороно считаем по первой записи amplitude_id (id логов идут обычно в порядке появления в БД)
# id лога превращаем в число
sign.amplitude_id = sign.amplitude_id.astype('int')
# сортируем и осталяем первую запись для дубликатов 
sign = sign.sort_values('amplitude_id').drop_duplicates('gp:USER_ID', keep='first')

# доп преобразовния
sign['MEDIA_SOURCE'] = sign['        gp:MEDIA_SOURCE']
sign.loc[sign['MEDIA_SOURCE'] == 'blog', 
         'MEDIA_SOURCE'] = 'Blog'
sign.loc[sign['MEDIA_SOURCE'] == '', 
         'MEDIA_SOURCE'] = 'unknown'


sign = sign[['amplitude_id','country','platform', 'start_version', 'version',
      'gp:COHORT_DAY','gp:COHORT_WEEK','gp:COHORT_MONTH','gp:COHORT_YEAR',
      'gp:IS_DEMO','MEDIA_SOURCE','gp:CAPMAIGN','gp:USER_ID']]


# Эффективность эксперимента

In [698]:
# 
total_users = (sign.groupby('platform')['gp:USER_ID'].nunique().reset_index()
 .rename(columns={'gp:USER_ID':'total_users'}))

paying_users = (sign[sign['gp:USER_ID'].isin(payments.Email)]
 .groupby('platform')['gp:USER_ID'].nunique().reset_index()
 .rename(columns={'gp:USER_ID':'paying_users'}))

tab_platform = total_users.merge(paying_users, on='platform')

tab_platform['conversion'] = round(tab_platform.paying_users /tab_platform.total_users, 4)


Общая конверсия в покупку


In [694]:
round(tab_platform.paying_users.sum()/tab_platform.total_users.sum(), 4)


0.038

Конверсия в покупку по платформам

In [699]:
tab_platform

,platform,total_users,paying_users,conversion
0,Android,4943,138,0.0279
1,iOS,1419,104,0.0733


Различие конверсии iOS vs Android

In [733]:
print('p-value по критерию Хи-квадрат:')

'%.14f' % chi2_contingency([[tab_platform.total_users[0]-tab_platform.paying_users[0],tab_platform.paying_users[0]],
                            [tab_platform.total_users[1]-tab_platform.paying_users[1],tab_platform.paying_users[1]]])[1]


p-value по критерию Хи-квадрат:


'0.00000000000001'

In [701]:
import scipy.stats as stats

print('p-value по точному критерию Фишера:')

'%.14f' % stats.fisher_exact([[tab_platform.total_users[0]-tab_platform.paying_users[0],tab_platform.paying_users[0]],
                            [tab_platform.total_users[1]-tab_platform.paying_users[1],tab_platform.paying_users[1]]])[1]


p-value по точному критерию Фишера:


'0.00000000000024'

Фильтруем версию по "start_version", когда сразу после регистрации юзер видел или не видел изменения

In [726]:
# проверка того, что нет юзеров, которые купили в 19 версии и это было до даты релиза
print(sign[(sign['start_version'] == '19') &
    (sign['gp:USER_ID'].isin(payments[payments['Payment Dt'] < '2020-12-19'].Email))
]['gp:USER_ID'].nunique())

0


In [715]:
# Статистика по тем, кто купил

# всего купили в 19 версии и при этом юзеры есть в выгрузке sign-ups 
buy_after_release = sign[(sign['start_version'] == '19') &
    (sign['gp:USER_ID'].isin(payments.Email))
]['gp:USER_ID'].nunique()

# & (sign.platform == 'iOS')
buy_after_release_iOS = sign[(sign['start_version'] == '19') &
    (sign['gp:USER_ID'].isin(payments.Email)) & (sign.platform == 'iOS')
]['gp:USER_ID'].nunique()

# & (sign.platform == 'Android')
buy_after_release_Android = sign[(sign['start_version'] == '19') &
    (sign['gp:USER_ID'].isin(payments.Email)) & (sign.platform == 'Android')
]['gp:USER_ID'].nunique()

# всего купили НЕ в 19 версии
buy_before_release = sign[(sign['start_version'] != '19') &
    (sign['gp:USER_ID'].isin(payments.Email)) 
]['gp:USER_ID'].nunique()

# & (sign.platform == 'iOS')
buy_before_release_iOS = sign[(sign['start_version'] != '19') &
    (sign['gp:USER_ID'].isin(payments.Email)) & (sign.platform == 'iOS')
]['gp:USER_ID'].nunique()

# & (sign.platform == 'Android')
buy_before_release_Android = sign[(sign['start_version'] != '19') &
    (sign['gp:USER_ID'].isin(payments.Email)) & (sign.platform == 'Android')
]['gp:USER_ID'].nunique()


# Статистика по всем юзерам

# все, кто использовал с 19 версии
total_19 = sign[(sign['start_version'] == '19') ]['gp:USER_ID'].nunique()

# & (sign.platform == 'Android')
total_19_Android = sign[(sign['start_version'] == '19') & (sign.platform == 'Android') ]['gp:USER_ID'].nunique()

# & (sign.platform == 'iOS')
total_19_iOS = sign[(sign['start_version'] == '19') & (sign.platform == 'iOS') ]['gp:USER_ID'].nunique()

# все, кто использовал НЕ с 19 версии
total_not_19 = sign[(sign['version'] != '19')
]['gp:USER_ID'].nunique()

# & (sign.platform == 'Android')
total_not_19_Android = sign[(sign['version'] != '19') & (sign.platform == 'Android')
]['gp:USER_ID'].nunique()

# & (sign.platform == 'iOS')
total_not_19_iOS = sign[(sign['version'] != '19') & (sign.platform == 'iOS')
]['gp:USER_ID'].nunique()


Сравнение общих конверсий ДО и ПОСЛЕ эксперимента

In [730]:
# данные
cols = ['user_filter','users','paying_users']
list_of_series = [pd.Series(['ДО эксперимента',total_not_19,buy_before_release],index=cols),
                  pd.Series(['ПОСЛЕ эксперимента', total_19,buy_after_release],index=cols)]
tab_total = pd.DataFrame(list_of_series, columns=cols)

tab_total['conversion'] = round(tab_total.paying_users/tab_total.users ,4)
tab_total

,user_filter,users,paying_users,conversion
0,ДО эксперимента,4638,181,0.0390
1,ПОСЛЕ эксперимента,1307,61,0.0467


In [717]:
print('p-value по критерию Хи-квадрат:')

chi2_contingency([[total_19-buy_after_release,buy_after_release],
                  [total_not_19-buy_before_release,buy_before_release]])[1]


p-value по критерию Хи-квадрат:


0.24753952634241588

In [732]:
# данные
cols = ['user_filter','users','paying_users']
list_of_series = [pd.Series(['ДО эксперимента iOS',total_not_19_iOS,buy_before_release_iOS],index=cols), 
                  pd.Series(['ДО эксперимента Android',total_not_19_Android,buy_before_release_Android],index=cols), 
                  pd.Series(['ПОСЛЕ эксперимента iOS',total_19_iOS,buy_after_release_iOS],index=cols), 
                  pd.Series(['ПОСЛЕ эксперимента Android',total_19_Android,buy_after_release_Android],index=cols)]
                  
tab_platform_ver = pd.DataFrame(list_of_series, columns=cols)

tab_platform_ver['conversion'] = round(tab_platform_ver.paying_users/tab_platform_ver.users ,4)
tab_platform_ver


,user_filter,users,paying_users,conversion
0,ДО эксперимента iOS,1419,104,0.0733
1,ДО эксперимента Android,3219,77,0.0239
2,ПОСЛЕ эксперимента iOS,0,0,NaN
3,ПОСЛЕ эксперимента Android,1307,61,0.0467


In [720]:
# Android
from scipy.stats import chi2_contingency

print('p-value по критерию Хи-квадрат:')

'%.8f' % chi2_contingency([[total_19_Android-buy_after_release_Android,buy_after_release_Android],
                  [total_not_19_Android-buy_before_release_Android,buy_before_release_Android]])[1]


p-value по критерию Хи-квадрат:


'0.00008178'

In [721]:
# Android
import scipy.stats as stats

print('p-value по точному критерию Фишера:')

'%.8f' % stats.fisher_exact([[total_19_Android-buy_after_release_Android,buy_after_release_Android],
                  [total_not_19_Android-buy_before_release_Android,buy_before_release_Android]])[1]


p-value по точному критерию Фишера:


'0.00011706'

# Каналы привлечения и юнит-экономика

In [764]:
# данные по всем платформам
sources = (sign['MEDIA_SOURCE'].value_counts().reset_index()
 .rename(columns={'index':'source', 'MEDIA_SOURCE':'total_users'})
 .merge((sign[(sign['gp:USER_ID'].isin(payments.Email))]['MEDIA_SOURCE']
         .value_counts().reset_index()
         .rename(columns={'index':'source', 'MEDIA_SOURCE':'paying_users'})), how='left',on='source')
)

sources = sources.fillna(0)
sources.paying_users = sources.paying_users.astype('int')

sources['share_of_source'] = round(sources.total_users / sources.total_users.sum(), 3)

sources['conversion'] = round(sources.paying_users / sources.total_users, 3)

sources_revenue = (sign.merge(payments, left_on='gp:USER_ID', right_on='Email')
 .groupby('MEDIA_SOURCE')['Amount'].sum().reset_index())


sources = sources.merge(sources_revenue, how='left', left_on='source', right_on='MEDIA_SOURCE').fillna(0)

sources.rename(columns={'Amount': 'revenue_by_source'}, inplace=True)

sources = sources[['source','total_users','share_of_source',
         'paying_users','conversion','revenue_by_source']]

sources['ARPU'] = round(sources.revenue_by_source / sources.total_users, 2)
sources['ARPPU'] = round(sources.revenue_by_source / sources.paying_users, 2)


In [778]:
sources.sort_values('ARPU',ascending=False)

,source,total_users,share_of_source,paying_users,conversion,revenue_by_source,ARPU,ARPPU
18,None,1,0.000,1,1.000,53.50,53.50,53.50
2,landing,259,0.041,33,0.127,8858.35,34.20,268.43
9,Blog,67,0.011,11,0.164,1574.64,23.50,143.15
5,QR_code,180,0.028,31,0.172,2532.93,14.07,81.71
3,landing_email,230,0.036,30,0.130,2300.94,10.00,76.70
4,Cindicator Website,215,0.034,18,0.084,1342.92,6.25,74.61
7,Cross_sale,142,0.022,5,0.035,846.36,5.96,169.27
1,unknown,2355,0.370,104,0.044,12437.53,5.28,119.59
11,Telegram Promo,24,0.004,2,0.083,125.00,5.21,62.50
10,Social_twitter,41,0.006,1,0.024,200.00,4.88,200.00


Сортируя по ARPU видим, что каналы с наивысшим доходом на привлеченного юзера: landing, Blog, QR_code, landing_email (для None слишком мало данных для оценки)

Но эти каналы занимают очень маленькую долю от всего трафика, нужно расширять воронку для этих каналов.


In [741]:
# Источники по всем платформам
go.Figure(data=[go.Pie(labels=sources.source.head(16), values=sources.share_of_source.head(16))])


In [801]:
# данные по iOS
sources_iOS = (sign[sign.platform == 'iOS']['MEDIA_SOURCE'].value_counts().reset_index()
 .rename(columns={'index':'source', 'MEDIA_SOURCE':'total_users'})
 .merge((sign[(sign['gp:USER_ID'].isin(payments.Email)) & (sign.platform == 'iOS')]['MEDIA_SOURCE']
         .value_counts().reset_index()
         .rename(columns={'index':'source', 'MEDIA_SOURCE':'paying_users'})), how='left',on='source')
)

sources_iOS = sources_iOS.fillna(0)
sources_iOS.paying_users = sources_iOS.paying_users.astype('int')

sources_iOS['share_of_source'] = round(sources_iOS.total_users / sources_iOS.total_users.sum(), 3)

sources_iOS['conversion'] = round(sources_iOS.paying_users / sources_iOS.total_users, 3)

sources_iOS_revenue = (sign[sign.platform == 'iOS'].merge(payments, left_on='gp:USER_ID', right_on='Email')
 .groupby('MEDIA_SOURCE')['Amount'].sum().reset_index())


sources_iOS = sources_iOS.merge(sources_iOS_revenue, how='left', left_on='source', right_on='MEDIA_SOURCE').fillna(0)

sources_iOS.rename(columns={'Amount': 'revenue_by_source'}, inplace=True)

sources_iOS = sources_iOS[['source','total_users','share_of_source',
         'paying_users','conversion','revenue_by_source']]

sources_iOS['ARPU'] = round(sources_iOS.revenue_by_source / sources_iOS.total_users, 2)
sources_iOS['ARPPU'] = round(sources_iOS.revenue_by_source / sources_iOS.paying_users, 2)


In [802]:
sources_iOS.sort_values('ARPU',ascending=False).head(12)

,source,total_users,share_of_source,paying_users,conversion,revenue_by_source,ARPU,ARPPU
3,landing,125,0.088,20,0.160,7594.43,60.76,379.72
16,None,1,0.001,1,1.000,53.50,53.50,53.50
8,Blog,31,0.022,7,0.226,1307.56,42.18,186.79
5,QR_code,92,0.065,10,0.109,1382.93,15.03,138.29
0,unknown,587,0.414,41,0.070,5390.62,9.18,131.48
4,landing_email,114,0.080,12,0.105,634.46,5.57,52.87
6,Cindicator Website,89,0.063,7,0.079,465.42,5.23,66.49
1,Apple Search Ads,170,0.120,4,0.024,507.95,2.99,126.99
7,Cross_sale,44,0.031,1,0.023,52.80,1.20,52.80
2,bidease_int,125,0.088,1,0.008,51.09,0.41,51.09


In [803]:
# данные по Android

sources_Android = (sign[sign.platform == 'Android']['MEDIA_SOURCE'].value_counts().reset_index()
 .rename(columns={'index':'source', 'MEDIA_SOURCE':'total_users'})
 .merge((sign[(sign['gp:USER_ID'].isin(payments.Email)) & (sign.platform == 'Android')]['MEDIA_SOURCE']
         .value_counts().reset_index()
         .rename(columns={'index':'source', 'MEDIA_SOURCE':'paying_users'})), how='left',on='source')
)

sources_Android = sources_Android.fillna(0)
sources_Android.paying_users = sources_Android.paying_users.astype('int')

sources_Android['share_of_source'] = round(sources_Android.total_users / sources_Android.total_users.sum(), 3)

sources_Android['conversion'] = round(sources_Android.paying_users / sources_Android.total_users, 3)

sources_Android_revenue = (sign[sign.platform == 'Android'].merge(payments, left_on='gp:USER_ID', right_on='Email')
 .groupby('MEDIA_SOURCE')['Amount'].sum().reset_index())


sources_Android = sources_Android.merge(sources_Android_revenue, how='left', left_on='source', right_on='MEDIA_SOURCE').fillna(0)

sources_Android.rename(columns={'Amount': 'revenue_by_source'}, inplace=True)

sources_Android = sources_Android[['source','total_users','share_of_source',
         'paying_users','conversion','revenue_by_source']]

sources_Android['ARPU'] = round(sources_Android.revenue_by_source / sources_Android.total_users, 2)
sources_Android['ARPPU'] = round(sources_Android.revenue_by_source / sources_Android.paying_users, 2)


In [804]:
sources_Android.sort_values('ARPU',ascending=False).head(10)

,source,total_users,share_of_source,paying_users,conversion,revenue_by_source,ARPU,ARPPU
4,landing_email,116,0.023,18,0.155,1666.48,14.37,92.58
7,QR_code,88,0.018,21,0.239,1150.00,13.07,54.76
2,landing,134,0.027,13,0.097,1263.92,9.43,97.22
5,Cross_sale,98,0.020,4,0.041,793.56,8.10,198.39
9,Social_twitter,25,0.005,1,0.040,200.00,8.00,200.00
8,Blog,36,0.007,4,0.111,267.08,7.42,66.77
3,Cindicator Website,126,0.025,11,0.087,877.50,6.96,79.77
10,Telegram Promo,19,0.004,2,0.105,125.00,6.58,62.50
1,unknown,1768,0.358,63,0.036,7046.91,3.99,111.86
0,bidease_int,2421,0.490,1,0.000,51.12,0.02,51.12


Показатели юнит-экономики

In [818]:
# данные
cols = ['platform_filter','ARPU','ARPPU','revenue']
list_of_series = [pd.Series(['По всем платформам',
                             round(sources.revenue_by_source.sum() / sources.total_users.sum(), 2),
                             round(sources.revenue_by_source.sum() / sources.paying_users.sum(), 2), 
                             (sources_iOS.revenue_by_source.sum() + sources_Android.revenue_by_source.sum())],index=cols), 
                  pd.Series(['Android',
                             round(sources_Android.revenue_by_source.sum() / sources_Android.total_users.sum(),2),
                             round(sources_Android.revenue_by_source.sum() / sources_Android.paying_users.sum(),2), 
                             (sources_Android.revenue_by_source.sum())],index=cols), 
                  pd.Series(['iOS',
                             round(sources_iOS.revenue_by_source.sum() / sources_iOS.total_users.sum(),2),
                             round(sources_iOS.revenue_by_source.sum() / sources_iOS.paying_users.sum(),2), 
                             (sources_iOS.revenue_by_source.sum())],index=cols)]
                  
tab_unit = pd.DataFrame(list_of_series, columns=cols)

tab_unit['revenue_share'] = round(tab_unit.revenue/tab_unit.revenue[0], 2)

tab_unit


,platform_filter,ARPU,ARPPU,revenue,revenue_share
0,По всем платформам,4.85,127.61,30882.33,1.00
1,Android,2.72,97.40,13441.57,0.44
2,iOS,12.29,167.70,17440.76,0.56
